# Лабораторная работа №1
### Выполнили:
#### Соколов Никита, Минеев Кирилл, Соловьёв Дмитрий, Семенов Евгений

# Задание 1. Шифр Хилла.

### Установка необходимых библиотек:

In [1]:
!pip install numpy sympy

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
from random import randint
import math

In [3]:
alphabet = "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ "
print(len(alphabet))

34


### Ценное сообщение из 12 символов:

In [4]:
message = "Я ЛЮБЛЮ ХЛЕБ"

### 3 матрицы-ключа:

In [5]:
def create_matrix(key_size):
    while True:
        matrix = np.random.randint(0, 33, size=(key_size, key_size))
        if math.gcd(round(np.linalg.det(matrix)), len(alphabet)) == 1:
            try:
                inv_matrix = np.linalg.inv(matrix)
                if np.allclose(inv_matrix, np.round(inv_matrix)):
                    return matrix
            except np.linalg.LinAlgError:
                continue

key_matrix2 = create_matrix(2)
key_matrix3 = create_matrix(3)
key_matrix4 = create_matrix(4)
print(f"Матрица 2x2:\n{key_matrix2}")
print(f"Матрица 3x3:\n{key_matrix3}")
print(f"Матрица 4x4:\n{key_matrix4}")

Матрица 2x2:
[[14 15]
 [29 31]]
Матрица 3x3:
[[11 17 23]
 [31  6 24]
 [19  5 16]]
Матрица 4x4:
[[ 0 19 15 25]
 [17 13 15 30]
 [13 17 28 32]
 [19  0 25 14]]


### Убедимся, что определители матриц не имеют общих делителей с длиной алфавита:

In [6]:
det_matrix2 = round(np.linalg.det(key_matrix2))
det_matrix3 = round(np.linalg.det(key_matrix3))
det_matrix4 = round(np.linalg.det(key_matrix4))

print(f"Определители для матриц: 2x2: {det_matrix2}, 3x3: {det_matrix3}, 4x4: {det_matrix4}")
print(f"Условие выполнено: {all(np.gcd(det_matrix_i, len(alphabet)) == 1 for det_matrix_i in (det_matrix2, det_matrix3, det_matrix4))}")

Определители для матриц: 2x2: -1, 3x3: -1, 4x4: -1
Условие выполнено: True


### Зашифруем сообщение с помощью каждого из ключей:

In [7]:
def encrypt(key_matrix, msg):
    message_indices = [alphabet.index(char) for char in msg]
    key_size = len(key_matrix)
    
    while len(message_indices) % key_size != 0:
        message_indices.append(alphabet.index(" "))

    encrypted_indices = []
    for i in range(0, len(message_indices), key_size):
        block = np.array(message_indices[i : i + key_size])
        encrypted_block = np.dot(key_matrix, block) % len(alphabet)
        encrypted_indices.extend(encrypted_block)
        
    return "".join([alphabet[i] for i in encrypted_indices])

enc2 = encrypt(key_matrix2, message)
enc3 = encrypt(key_matrix3, message)
enc4 = encrypt(key_matrix4, message)
print(f"Сообщения:\n1. {enc2}\n2. {enc3}\n3. {enc4}")

Сообщения:
1. ШМФРЧЪКСЛЧРЁ
2.  ПМХЮДНФСВСЮ
3. СИЪПХЭ ЯХЦПМ


### Теперь расшифруем сообщения: 

In [8]:
def decrypt(key_matrix, enc):
    inv_key_matrix = np.round(np.linalg.inv(key_matrix)).astype(int)
    return encrypt(inv_key_matrix, enc)
    
dec2 = decrypt(key_matrix2, enc2)
dec3 = decrypt(key_matrix3, enc3)
dec4 = decrypt(key_matrix4, enc4)

print(f"Сообщения:\n1. {dec2}\n2. {dec3}\n3. {dec4}")

Сообщения:
1. Я ЛЮБЛЮ ХЛЕБ
2. Я ЛЮБЛЮ ХЛЕБ
3. Я ЛЮБЛЮ ХЛЕБ


### Сымитируем вредоносное вмешательство в зашифрованные сообщения:

In [9]:
def hack(enc):
    indexes = []
    chars = []
    listed_mes = list(enc)

    while len(indexes) != 3:
        index = np.random.randint(0, len(enc))
        char = alphabet[np.random.randint(0, len(alphabet))]
        if (index not in indexes) and (char != enc[index]):
            indexes.append(index)
            chars.append(char)

    for i in range(0, 3):
        listed_mes[indexes[i]] = chars[i]

    return "".join(listed_mes)


hacked_enc2, hacked_enc3, hacked_enc4 = (hack(enc_i) for enc_i in (enc2, enc3, enc4))
print(f"Сообщения:\n1. {hacked_enc2}\n2. {hacked_enc3}\n3. {hacked_enc4}")

Сообщения:
1. ШМФРЧХКПЛБРЁ
2.  ЭМХЮДЩФСВБЮ
3. САЪПХЭ ЪХЦП 


### Расшифруем получившиеся сообщения: 

In [10]:
hacked_dec2, hacked_dec3, hacked_dec4 = (decrypt(key_matrix_i, enc_i) for key_matrix_i, enc_i in zip((key_matrix2, key_matrix3, key_matrix4), (hacked_enc2, hacked_enc3, hacked_enc4)))

print(f"Сообщения:\n1. {hacked_dec2}\n2. {hacked_dec3}\n3. {hacked_dec4}")

Сообщения:
1. Я ЛЮЫНБЪРЫЕБ
2. УОАЮБЛМГЁЬХБ
3. ОУКРЫШБСПЫЦЪ


# Задание 2. Взлом шифра Хилла.

### Ключ размера 2x2:

In [11]:
matrix_key_hidden = create_matrix(2)
print(matrix_key_hidden)

[[10  3]
 [ 3  1]]


### 2 сообщения из 12 символов:

In [12]:
message1 = "Я ЛЮБЛЮ БОРЩ"
message2 = "Я ЛЮБЛЮ ХЛЕБ"

### Шифруем 2 сообщения: 

In [13]:
enc1 = encrypt(matrix_key_hidden, message1)
enc2 = encrypt(matrix_key_hidden, message2)
print(f"Сообщения:\n1. {enc1}\n2. {enc2}")

Сообщения:
1. КЪИ ЛОБЧФСЙИ
2. КЪИ ЛОБЧСЙТП


### Забыли одно исходное сообщение:

In [14]:
message1 = None

### Восстанавливаем ключ по исходному и шифрованному сообщению:

In [15]:
encr_message_indices = [alphabet.index(char) for char in enc2]
orig_message_indices = [alphabet.index(char) for char in message2]
matrix_key = None
flag = False

for i in range(0, len(orig_message_indices)):
    for j in range(i, len(orig_message_indices)-3):
        orig_block = np.array([orig_message_indices[i:i+2], orig_message_indices[j+2:j+4]]).T
        encr_block = np.array([encr_message_indices[i:i+2], encr_message_indices[j+2:j+4]]).T
        for x in range(0, len(alphabet)):
            if ((x * np.linalg.det(orig_block)) % len(alphabet)) == 1:
                break
        inv_orig_block = (x * np.array([[orig_block[1][1], -1* orig_block[1][0]], [-1*orig_block[0][1], orig_block[0][0]]]).T) % len(alphabet)
        if np.allclose(inv_orig_block, np.round(inv_orig_block)):
            matrix_key = (np.dot(encr_block, inv_orig_block) % len(alphabet)).astype(int)
            flag = (encrypt(matrix_key, message2) == enc2)
        if flag:
            break
    if flag:
        break
if matrix_key is not None:
    print(matrix_key)
    
else:
    print("Не получилось расшифровать, так как матрица фразы никак не инвертируется")

[[10  3]
 [ 3  1]]


### Расшифровываем забытое сообщение:

In [16]:
decrypt(matrix_key, enc1)

'Я ЛЮБЛЮ БОРЩ'

# Задание 3. Код Хэмминга

### Русский алфавит:


In [17]:
russian_alphabet = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
n = len(russian_alphabet)
n

32

### Присвоим каждой букве пятибитовый двоичный номер:

In [18]:
def convert_to_five_bit_number(number):
    bin_number = ""
    while number > 0:
        bin_number += str(number % 2)
        number //= 2
    return "0" * (5 - len(bin_number)) + bin_number[::-1]


russian_bin_alphabet = {russian_alphabet[i]: convert_to_five_bit_number(i) for i in range(n)}
russian_bin_alphabet

{'а': '00000',
 'б': '00001',
 'в': '00010',
 'г': '00011',
 'д': '00100',
 'е': '00101',
 'ж': '00110',
 'з': '00111',
 'и': '01000',
 'й': '01001',
 'к': '01010',
 'л': '01011',
 'м': '01100',
 'н': '01101',
 'о': '01110',
 'п': '01111',
 'р': '10000',
 'с': '10001',
 'т': '10010',
 'у': '10011',
 'ф': '10100',
 'х': '10101',
 'ц': '10110',
 'ч': '10111',
 'ш': '11000',
 'щ': '11001',
 'ъ': '11010',
 'ы': '11011',
 'ь': '11100',
 'э': '11101',
 'ю': '11110',
 'я': '11111'}

### Интересное слово из 4 букв:

In [19]:
word = "шифр"

### Закодируем двоичным кодом:

In [20]:
encrypted_bin_word = "".join([russian_bin_alphabet[letter] for letter in list(word)])
encrypted_bin_word

'11000010001010010000'

### Составим матрицы G и H:

In [21]:
G = np.array([[1, 1, 0, 1], [1, 0, 1, 1], [1, 0, 0, 0], [0, 1, 1, 1], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
H = np.array([[1, 0, 1, 0, 1, 0, 1], [0, 1, 1, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1, 1]])

* Матрица G - это матрица-генератор кодов, которая преобразует информационное сообщение, представленное вектором размерности (4,1), в кодовое слово размерности (7,1). При этом она добавляет три дополнительных бита четности. Поэтому размер матрицы G составляет (4,7).
* Матрица H используется для проверки целостности закодированного сообщения. Она анализирует кодовое слово размерности (7,1) и возвращает вектор размерности (3,1), который указывает на наличие ошибок. Если кодовое слово не повреждено, результат будет нуль-вектором; если присутствует ошибка, матрица H укажет на позицию испорченного бита. Соответственно, размер матрицы H составляет (3,7).
* Можно сделать вывод, что ker H = Im G

### Закодируем слово с помощью матрицы G:

In [22]:
encoded_word = ''
for i in range(0, len(encrypted_bin_word), 4):
    p = np.array([int(c) for c in encrypted_bin_word[i : i + 4]])
    x = G.dot(p) % 2
    encoded_word += ''.join([str(i) for i in x])

encoded_word

'01111000101010010101000110010000000'

### Сымитируем вредоносное вмешательство:

In [23]:
broken_words = []
broken1 = list(encoded_word)
for _ in range(1):
    i = randint(0, len(broken1) - 1)
    broken1[i] = '1' if broken1[i] == '0' else '0'
broken1 = ''.join(broken1)
broken_words.append(broken1)

broken2 = list(encoded_word)
for _ in range(2):
    i = randint(0, len(broken2) - 1)
    broken2[i] = '1' if broken2[i] == '0' else '0'
broken2 = ''.join(broken2)
broken_words.append(broken2)

broken3 = list(encoded_word)
for _ in range(3):
    i = randint(0, len(broken3) - 1)
    broken3[i] = '1' if broken3[i] == '0' else '0'
broken3 = ''.join(broken3)
broken_words.append(broken3)

broken4 = list(encoded_word)
for _ in range(4):
    i = randint(0, len(broken4) - 1)
    broken4[i] = '1' if broken4[i] == '0' else '0'
broken4 = ''.join(broken4)
broken_words.append(broken4)

### Находим и исправляем ошибки с помощью матрицы H и декодируем сообщение:

In [24]:
R = np.array([[0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1]])
word_bins = []
for broken in broken_words:
    word_bin = ''
    for i in range(0, len(broken), 7):
        r = np.array([int(c) for c in broken[i : i + 7]])
        z = H.dot(r) % 2
        print(z)
        if sum(z) != 0:
            idx = int(''.join([str(n) for n in z[::-1]]), 2) - 1
            r[idx] = 1 if r[idx] == 0 else 0
        p = R.dot(r) % 2
        word_bin += ''.join([str(i) for i in p])
    print('---')
    word_bins.append(word_bin)

[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[1 1 1]
---
[0 0 0]
[0 0 0]
[0 0 0]
[0 1 0]
[1 0 1]
---
[1 1 1]
[0 0 1]
[0 0 0]
[1 1 1]
[0 0 0]
---
[0 0 0]
[1 0 0]
[1 1 1]
[1 1 0]
[0 1 0]
---


### Переводим полученный результат в слово из 4-х букв

In [25]:
for word_bin in word_bins:
    result = ''
    for i in range(0, len(word_bin), 5):
        result += russian_alphabet[list(russian_bin_alphabet.values()).index(word_bin[i : i + 5])]
    print(result)

шифр
шифр
шифр
шифр


### Вывод: 
#### В данной лабораторной работе мы узнали о разных видах шифров. Научились шифровать и с/без ключа расшифровывать сообщения. Поняли как работает шифр Хилла и код Хэмминга